In [1]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
import re
from bs4 import BeautifulSoup

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mourya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# EDA

In [2]:
import pandas as pd
df = pd.read_csv('seek_df.csv')
df.head()

,pageurl,crawl_timestamp,job_title,category,company_name,city,post_date,job_description,job_type,job_board,geo,state,salary_offered
0,https://www.seek.com.au/job/36028685,2018-04-20 03:57:04 +0000,Package Lead - Pipeline Installation,"Mining, Resources & Energy",FIRCROFT AUSTRALIA PTY LTD,Perth,2018-04-19T05:41:52Z,The Role: General Execution Accountable for sa...,Contract/Temp,seek,AU,NaN,NaN
1,https://www.seek.com.au/job/36028693,2018-04-20 03:52:46 +0000,Department Manager - Bakery - Campbelltown Region,Retail & Consumer Products,Coles,Sydney,2018-04-19T05:42:19Z,The role As a Coles Bakery Manager you will: w...,Full Time,seek,AU,South West & M5 Corridor,NaN
2,https://www.seek.com.au/job/36027858,2018-04-20 04:12:01 +0000,Freight Handler,"Manufacturing, Transport & Logistics",Zoom Recruitment & Training,Sydney,2018-04-19T04:51:51Z,Our client is a leader within the Transport / ...,Casual/Vacation,seek,AU,Parramatta & Western Suburbs,$34 - $39 p.h.
3,https://www.seek.com.au/job/36028687,2018-04-20 03:51:23 +0000,Warehouse Assistant,"Manufacturing, Transport & Logistics",Private Advertiser,Bundaberg & Wide Bay Burnett,2018-04-19T05:42:02Z,One of our Clients is looking for a Warehouse ...,Full Time,seek,AU,NaN,NaN
4,https://www.seek.com.au/job/36026414,2018-04-20 04:14:42 +0000,HR Truck Subcontractors,"Manufacturing, Transport & Logistics",Sands Fridge Lines,Perth,2018-04-19T03:22:40Z,Sands Fridge Lines WA is seeking Subcontractor...,Contract/Temp,seek,AU,NaN,NaN


In [3]:
df.shape

(20030, 13)

In [4]:
#check for missing values
df.isnull().sum()

pageurl                0
crawl_timestamp        0
job_title              0
category               0
company_name           0
city                   0
post_date              0
job_description    15305
job_type               0
job_board              0
geo                    0
state               6544
salary_offered     14286
dtype: int64

In [5]:
df["category"].value_counts()

Trades & Services                         2056
Information & Communication Technology    1911
Healthcare & Medical                      1535
Manufacturing, Transport & Logistics      1523
Administration & Office Support           1250
Hospitality & Tourism                     1141
Accounting                                1084
Sales                                      961
Construction                               894
Government & Defence                       810
Retail & Consumer Products                 766
Education & Training                       723
Engineering                                665
Call Centre & Customer Service             534
Community Services & Development           526
Mining, Resources & Energy                 492
Banking & Financial Services               468
Marketing & Communications                 461
Human Resources & Recruitment              448
Legal                                      385
Real Estate & Property                     352
Design & Arch

In [6]:
print("Unique values in job_type column:", df['job_type'].unique())

Unique values in job_type column: ['Contract/Temp' 'Full Time' 'Casual/Vacation' 'Part Time']


In [7]:
#new_df contains all the rows without the missing values from job_description
new_df = df[df['job_description'].notna()]

In [8]:
new_df.shape

(4725, 13)

In [9]:
new_df["category"].value_counts()

Trades & Services                         487
Information & Communication Technology    438
Healthcare & Medical                      388
Manufacturing, Transport & Logistics      354
Administration & Office Support           305
Sales                                     240
Accounting                                232
Hospitality & Tourism                     228
Construction                              197
Retail & Consumer Products                196
Education & Training                      171
Engineering                               165
Community Services & Development          164
Government & Defence                      163
Call Centre & Customer Service            121
Banking & Financial Services              120
Mining, Resources & Energy                118
Human Resources & Recruitment             103
Marketing & Communications                103
Real Estate & Property                     99
Legal                                      85
Design & Architecture             

In [10]:
#df["job_description"].fillna("No_Description", inplace = True)

# Helper Functions

In [11]:
stop_words = set(stopwords.words('english'))
print(stop_words)

{'shan', 'all', 'is', "don't", 'which', 'your', 'off', 'was', 'hers', 'had', 'through', "won't", 'weren', 'over', 'on', 'by', 'why', 'mightn', 'the', 'then', 'same', 'were', 'wasn', 'between', 'd', 'mustn', 'those', 'these', 'll', 'doesn', 'for', 'yours', 't', 'his', 'are', 'until', 'an', 'some', 'their', 'both', 'wouldn', 'only', 'will', 'after', 'in', "isn't", 'nor', 'other', 'under', "couldn't", 'has', 'but', 'before', 'here', 're', "shan't", 'am', 'as', 'now', 'most', 'don', 'should', 'that', 'hadn', 'so', 've', 'my', 'of', 'myself', 'while', 'such', "aren't", 'i', "wasn't", 'haven', 'aren', 'it', 'who', "that'll", 'have', 'when', "hadn't", 'themselves', "doesn't", 'her', 'didn', 'been', 'o', 'how', 'can', 'whom', 'and', 'more', 'what', 'too', 'isn', 'again', "needn't", 'yourself', 'he', 'she', 'below', 'me', "should've", 'ma', 'couldn', 'about', 'into', 'needn', 'once', 'shouldn', 'a', "wouldn't", "you're", 'himself', 'if', 'out', 'yourselves', 'against', 'did', 'from', 'you', "di

In [12]:
print("List of punctuations: " + string.punctuation)

#Clean column by removing punctuations and stopwords
def clean_column(text):
    '''
    #Remove punctuations and stop words
    words_array = return a character if the character is not in !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
    no_stop_words_array = return word if it is not a stopword
    '''
    words_array = "".join([char for char in text if char not in string.punctuation])

    tokens = re.split('\W+', words_array)

    no_stop_words_array = [word for word in tokens if word not in stop_words]

    sentence = ' '.join(no_stop_words_array)
    
    return sentence

List of punctuations: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [13]:
#Search for a keyword from text. If present return true, else return false
def key_word_search(text, key_word):
    if re.findall(key_word.lower(), text.lower()):
        return True
    else:
        return False

# Data cleaning

In [14]:
#create a new column with the cleaned text
new_df['cleaned_job_description'] = new_df['job_description'].apply(lambda x: clean_column(x.lower()))
new_df.head()

/Users/mourya/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,pageurl,crawl_timestamp,job_title,category,company_name,city,post_date,job_description,job_type,job_board,geo,state,salary_offered,cleaned_job_description
0,https://www.seek.com.au/job/36028685,2018-04-20 03:57:04 +0000,Package Lead - Pipeline Installation,"Mining, Resources & Energy",FIRCROFT AUSTRALIA PTY LTD,Perth,2018-04-19T05:41:52Z,The Role: General Execution Accountable for sa...,Contract/Temp,seek,AU,NaN,NaN,role general execution accountable safe effici...
1,https://www.seek.com.au/job/36028693,2018-04-20 03:52:46 +0000,Department Manager - Bakery - Campbelltown Region,Retail & Consumer Products,Coles,Sydney,2018-04-19T05:42:19Z,The role As a Coles Bakery Manager you will: w...,Full Time,seek,AU,South West & M5 Corridor,NaN,role coles bakery manager work within local co...
2,https://www.seek.com.au/job/36027858,2018-04-20 04:12:01 +0000,Freight Handler,"Manufacturing, Transport & Logistics",Zoom Recruitment & Training,Sydney,2018-04-19T04:51:51Z,Our client is a leader within the Transport / ...,Casual/Vacation,seek,AU,Parramatta & Western Suburbs,$34 - $39 p.h.,client leader within transport distribution in...
3,https://www.seek.com.au/job/36028687,2018-04-20 03:51:23 +0000,Warehouse Assistant,"Manufacturing, Transport & Logistics",Private Advertiser,Bundaberg & Wide Bay Burnett,2018-04-19T05:42:02Z,One of our Clients is looking for a Warehouse ...,Full Time,seek,AU,NaN,NaN,one clients looking warehouse assistant join t...
4,https://www.seek.com.au/job/36026414,2018-04-20 04:14:42 +0000,HR Truck Subcontractors,"Manufacturing, Transport & Logistics",Sands Fridge Lines,Perth,2018-04-19T03:22:40Z,Sands Fridge Lines WA is seeking Subcontractor...,Contract/Temp,seek,AU,NaN,NaN,sands fridge lines wa seeking subcontractors h...


In [15]:
new_df['skill_word_present'] = new_df['cleaned_job_description'].apply(lambda x: key_word_search(x, 'look'))
new_df.head()

/Users/mourya/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,pageurl,crawl_timestamp,job_title,category,company_name,city,post_date,job_description,job_type,job_board,geo,state,salary_offered,cleaned_job_description,skill_word_present
0,https://www.seek.com.au/job/36028685,2018-04-20 03:57:04 +0000,Package Lead - Pipeline Installation,"Mining, Resources & Energy",FIRCROFT AUSTRALIA PTY LTD,Perth,2018-04-19T05:41:52Z,The Role: General Execution Accountable for sa...,Contract/Temp,seek,AU,NaN,NaN,role general execution accountable safe effici...,False
1,https://www.seek.com.au/job/36028693,2018-04-20 03:52:46 +0000,Department Manager - Bakery - Campbelltown Region,Retail & Consumer Products,Coles,Sydney,2018-04-19T05:42:19Z,The role As a Coles Bakery Manager you will: w...,Full Time,seek,AU,South West & M5 Corridor,NaN,role coles bakery manager work within local co...,True
2,https://www.seek.com.au/job/36027858,2018-04-20 04:12:01 +0000,Freight Handler,"Manufacturing, Transport & Logistics",Zoom Recruitment & Training,Sydney,2018-04-19T04:51:51Z,Our client is a leader within the Transport / ...,Casual/Vacation,seek,AU,Parramatta & Western Suburbs,$34 - $39 p.h.,client leader within transport distribution in...,False
3,https://www.seek.com.au/job/36028687,2018-04-20 03:51:23 +0000,Warehouse Assistant,"Manufacturing, Transport & Logistics",Private Advertiser,Bundaberg & Wide Bay Burnett,2018-04-19T05:42:02Z,One of our Clients is looking for a Warehouse ...,Full Time,seek,AU,NaN,NaN,one clients looking warehouse assistant join t...,True
4,https://www.seek.com.au/job/36026414,2018-04-20 04:14:42 +0000,HR Truck Subcontractors,"Manufacturing, Transport & Logistics",Sands Fridge Lines,Perth,2018-04-19T03:22:40Z,Sands Fridge Lines WA is seeking Subcontractor...,Contract/Temp,seek,AU,NaN,NaN,sands fridge lines wa seeking subcontractors h...,False


In [16]:
new_df['skill_word_present'].value_counts()

False    2577
True     2148
Name: skill_word_present, dtype: int64

In [18]:
a = new_df[:10]

In [19]:
a

,pageurl,crawl_timestamp,job_title,category,company_name,city,post_date,job_description,job_type,job_board,geo,state,salary_offered,cleaned_job_description,skill_word_present
0,https://www.seek.com.au/job/36028685,2018-04-20 03:57:04 +0000,Package Lead - Pipeline Installation,"Mining, Resources & Energy",FIRCROFT AUSTRALIA PTY LTD,Perth,2018-04-19T05:41:52Z,The Role: General Execution Accountable for sa...,Contract/Temp,seek,AU,NaN,NaN,role general execution accountable safe effici...,False
1,https://www.seek.com.au/job/36028693,2018-04-20 03:52:46 +0000,Department Manager - Bakery - Campbelltown Region,Retail & Consumer Products,Coles,Sydney,2018-04-19T05:42:19Z,The role As a Coles Bakery Manager you will: w...,Full Time,seek,AU,South West & M5 Corridor,NaN,role coles bakery manager work within local co...,True
2,https://www.seek.com.au/job/36027858,2018-04-20 04:12:01 +0000,Freight Handler,"Manufacturing, Transport & Logistics",Zoom Recruitment & Training,Sydney,2018-04-19T04:51:51Z,Our client is a leader within the Transport / ...,Casual/Vacation,seek,AU,Parramatta & Western Suburbs,$34 - $39 p.h.,client leader within transport distribution in...,False
3,https://www.seek.com.au/job/36028687,2018-04-20 03:51:23 +0000,Warehouse Assistant,"Manufacturing, Transport & Logistics",Private Advertiser,Bundaberg & Wide Bay Burnett,2018-04-19T05:42:02Z,One of our Clients is looking for a Warehouse ...,Full Time,seek,AU,NaN,NaN,one clients looking warehouse assistant join t...,True
4,https://www.seek.com.au/job/36026414,2018-04-20 04:14:42 +0000,HR Truck Subcontractors,"Manufacturing, Transport & Logistics",Sands Fridge Lines,Perth,2018-04-19T03:22:40Z,Sands Fridge Lines WA is seeking Subcontractor...,Contract/Temp,seek,AU,NaN,NaN,sands fridge lines wa seeking subcontractors h...,False
5,https://www.seek.com.au/job/36027190,2018-04-20 04:05:15 +0000,"Administrative Officer, Education unit",Administration & Office Support,ANZCA,Melbourne,2018-04-19T04:11:11Z,Maternity leave contract ending May 2019 Full ...,Contract/Temp,seek,AU,CBD & Inner Suburbs,NaN,maternity leave contract ending may 2019 full ...,True
6,https://www.seek.com.au/job/36028057,2018-04-20 03:51:41 +0000,Traffic Controller - Sunshine Coast,Construction,Evolution Traffic Control,Sunshine Coast,2018-04-19T05:04:32Z,"Opportunity to earn up to $65,000 per annum in...",Casual/Vacation,seek,AU,NaN,NaN,opportunity earn 65000 per annum including sup...,True
7,https://www.seek.com.au/job/36028164,2018-04-20 03:45:12 +0000,Mortgage Broker Assistant,Banking & Financial Services,Howitt Partners,Melbourne,2018-04-19T05:11:49Z,Small but busy and very successful mortgage br...,Part Time,seek,AU,CBD & Inner Suburbs,$20 - $29.99 per hour,small busy successful mortgage broking busines...,False
8,https://www.seek.com.au/job/36025868,2018-04-20 04:31:05 +0000,Manager Violence Abuse and Neglect,Healthcare & Medical,Hunter New England Local Health District,Melbourne,2018-04-19T02:38:34Z,Employment Type: Permanent Full Time Position ...,Full Time,seek,AU,CBD & Inner Suburbs,NaN,employment type permanent full time position c...,False
9,https://www.seek.com.au/job/36026088,2018-04-20 04:25:11 +0000,Mortgage Broker,Banking & Financial Services,Private Advertiser,Melbourne,2018-04-19T02:53:23Z,If you are in the mortgage brokering industry ...,Full Time,seek,AU,NaN,"$70,000 - $89,999 base plus commission",mortgage brokering industry want leads provide...,False


In [17]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer

ngram_vect = CountVectorizer(ngram_range=(2,2))
x_counts = ngram_vect.fit_transform(new_df['cleaned_job_description'])
print(ngram_vect.get_feature_names())
print(x_counts.shape)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

